In [1]:
from transformers import AutoTokenizer
from transformers import LlamaForCausalLM
import torch

device = torch.device("cuda")
ckpt = '/opt/Data/ModelWeight/IDEA-CCNL/Ziya-LLaMA-13B'

model = LlamaForCausalLM.from_pretrained(ckpt, torch_dtype=torch.float16, device_map="cuda:0", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(ckpt, use_fast=False)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [2]:
query="帮我写一份去深圳的旅游计划,按照7天的时长规划"
inputs = '<human>:' + query.strip() + '\n<bot>:'
      
input_ids = tokenizer(inputs, return_tensors="pt").input_ids.to(device)
generate_ids = model.generate(
            input_ids,
            max_new_tokens=1024, 
            do_sample = True, 
            top_p = 0.85, 
            temperature = 1.0, 
            repetition_penalty=1., 
            eos_token_id=2, 
            bos_token_id=1, 
            pad_token_id=0)
output = tokenizer.batch_decode(generate_ids)[0]
print(output)

<s> <human> :帮我写一份去深圳的旅游计划,按照7天的时长规划 <bot>  : 好的，以下是为期7天的深圳旅游计划：
猛虎山公园：参观著名的猛虎山公园，观赏壮丽的山景和瀑布。
世界之窗：参观世界之窗，了解不同国家和文化的风情。
深圳湾公园：参观深圳湾公园，享受美丽的海滩和休闲娱乐设施。
华强北：参观华强北电子市场，购买各种电子产品和数码产品。
东海岸：前往东海岸，参观深圳市最美的海滨景观之一。
莲花山公园：参观莲花山公园，欣赏美丽的花园和湖泊景观。
深圳市民中心：参观深圳市民中心，了解深圳市的历史和发展。</s>
